[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter()

세션 ID 설정하여, 대화 진행합니다. (muti-turn conversation)
LLM 모델이 과거 대화를 알지 (기억) 못하는 상황을 만드세요.

In [3]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)


## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


## 6. trim 설정
trimmer = trim_messages(
    max_tokens = 565,      ## 최대 토큰 수
    strategy = 'last',     ## 최근 메시지부터 시작해서 자르기
    token_counter = llm,   ## 모델 기준으로 토큰 수 계산
    include_system= True,  ## 시스템 메시지 포함
    allow_partial= False,  ## 메시지 자르기 제한
    start_on= 'human',     ## human 메시지부터 자르기 시작
)


## 6. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history= itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
)

## 7. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


## 9. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 10. 실행

while True:
    query = input('육아휴직과 관련된 질문하세요 >> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'law': law, 'query': query},
        config={'configurable': {'session_id': 'abc'}}
    )

    print('\n' + '*' * 50 + '\n')

육아휴직의 기간은 1년 이내로 합니다. 다만, 다음 각 호의 어느 하나에 해당하는 근로자의 경우 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다.

1. 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우의 부 또는 모
2. 「한부모가족지원법」 제4조제1호의 부 또는 모
3. 고용노동부령으로 정하는 장애아동의 부 또는 모

이는 남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 제19조 제2항에 명시되어 있습니다.
**************************************************

육아휴직의 대상자는 임신 중인 여성 근로자와 만 8세 이하 또는 초등학교 2학년 이하의 자녀(입양한 자녀를 포함)를 양육하는 근로자입니다. 이러한 근로자가 육아휴직을 신청하는 경우, 사업주는 이를 허용해야 합니다. 이는 남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 제19조 제1항에 명시되어 있습니다.
**************************************************



In [5]:
for message in get_history('abc').messages:
    print(f'{[message.type.upper()]}: {message.content}\n')

['HUMAN']: 육아휴직 기간은?

['AI']: 육아휴직의 기간은 1년 이내로 합니다. 다만, 다음 각 호의 어느 하나에 해당하는 근로자의 경우 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다.

1. 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우의 부 또는 모
2. 「한부모가족지원법」 제4조제1호의 부 또는 모
3. 고용노동부령으로 정하는 장애아동의 부 또는 모

이는 남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 제19조 제2항에 명시되어 있습니다.

['HUMAN']: 육아휴직 대상자는?

['AI']: 육아휴직의 대상자는 임신 중인 여성 근로자와 만 8세 이하 또는 초등학교 2학년 이하의 자녀(입양한 자녀를 포함)를 양육하는 근로자입니다. 이러한 근로자가 육아휴직을 신청하는 경우, 사업주는 이를 허용해야 합니다. 이는 남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 제19조 제1항에 명시되어 있습니다.

